# MOTIFS

## Motifs Probabilísticos 

Motifs probabilísticos são padrões em sequências biológicas, como DNA ou proteínas, modelados com base em probabilidades. Eles representam a probabilidade de ocorrência de diferentes nucleotídeos ou aminoácidos em cada posição de um padrão. Estes são geralmente expressos por meio de PWM (Matriz de Probabilidade Ponderada) ou por PSSM (Matriz de Pontuação de Posição Específica). Os motifs proporcionam uma representação mais realista da variabilidade biológica.

Na área de bioinformática, esses motifs podem representar padrões de nucleótidos em sequências de DNA, sendo úteis para identificar regiões conservadas nesses ácidos nucleicos. Revelando-se como ferramentas indispensaveis para compreender a regulação genética e identificar características fundamentais em estudos genômicos.

### PWM (Position Weight Matrix)

In [1]:
def pwm(seqs: list[str], pseudo: float = 0) -> list[dict[str, float]]:
    """
    Calcula a matriz PWM (Matriz de Peso e Posição) para um conjunto de sequências

    Parâmetros:
    
    seqs : list[str]
        Recebe uma lista de strings que representam as sequências

    pseudo : float
        Recebe um valor opcional, pseudo, que em caso de omissão é = 0

    Retorna:
    
    pwm_matrix : list[dict[str, float]]
        Retorna uma *lista de dicionários*, onde cada *dicionário* terá uma chave no formato de *string*
        e um valor no formato de *float*

    Levanta:
    AssertError
        Caso a lista tenha sequências inválidas
    """

    # Valida cada caractere em cada posição das sequências
    for seq in seqs:
        for b in seq:
            assert b.isalpha(), f'Caracter {b} na sequência {seq} não é uma letra!'

    # Calcula a matriz PWM
    pwm_matrix = [{b: (sum(1 for seq in seqs if seq[i] == b) + pseudo) / (len(seqs) + pseudo * len(set(''.join(seqs))))
                   for b in set(''.join(seqs))}
                  for i in range(len(seqs[0]))]

    # Retorna a matriz PWM
    return pwm_matrix

Exemplo:

In [2]:
pip install tabulate

Note: you may need to restart the kernel to use updated packages.


In [14]:
# Sequências de DNA fornecidas para a função pwm
seqs = ['ATTG', 'ATCG', 'ATTC', 'ACTC']

# Chamada da função pwm com as sequências e o valor pseudo
resultado = pwm(seqs, pseudo=1)

# Impressão da matriz formatada com o módulo tabulate
from tabulate import tabulate

# Cabeçalhos para a tabela, incluindo "Base" e as posições numeradas
headers = ["Base"] + [f"Posição {i+1}" for i in range(len(seqs[0]))]

# Construção da tabela usando uma compreensão de lista 
table = [[base] + [round(resultado[i][base], 3) for i in range(len(resultado))] for base in set(''.join(seqs))]

# Impressão da tabela usando a função tabulate
print(tabulate(table, headers))

Base      Posição 1    Posição 2    Posição 3    Posição 4
------  -----------  -----------  -----------  -----------
C             0.125        0.25         0.25         0.375
A             0.625        0.125        0.125        0.125
T             0.125        0.5          0.5          0.125
G             0.125        0.125        0.125        0.375


Testes:

In [4]:
import unittest

class TestPWM(unittest.TestCase):

    def test_valid_sequences(self):
        seqs = ['ATTG', 'ATCG', 'ATTC', 'ACTC']
        pseudo = 1
        result = pwm(seqs, pseudo)
        expected_result = [
            {'A': 0.625, 'C': 0.125, 'G': 0.125, 'T': 0.125},
            {'A': 0.125, 'C': 0.25, 'G': 0.125, 'T': 0.5},
            {'A': 0.125, 'C': 0.25, 'G': 0.125, 'T': 0.5},
            {'A': 0.125, 'C': 0.375, 'G': 0.375, 'T': 0.125}
        ]
        self.assertEqual(result, expected_result)

    def test_empty_sequence(self):
        seqs = ['ATTG', 'ATCG', '', 'ACTC']  # Sequencia vazia
        pseudo = 1
        with self.assertRaises(AssertionError):
            pwm(seqs, pseudo)

    
    def test_special_characters(self):
        with self.assertRaises(AssertionError):
            pwm(['ATTG', 'ATCG', 'ATTC', 'ACT$'], pseudo=1)  # Sequencia invalida para carateres especiais
            
    
# Executar os testes
result = unittest.TextTestRunner().run(unittest.TestLoader().loadTestsFromTestCase(TestPWM))

# Verificar que falhas houveram nos testes
if not result.wasSuccessful():
    for failure in result.failures:
        print(failure)

E..
ERROR: test_empty_sequence (__main__.TestPWM.test_empty_sequence)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "C:\Users\tiago\AppData\Local\Temp\ipykernel_2872\277277751.py", line 21, in test_empty_sequence
    pwm(seqs, pseudo)
  File "C:\Users\tiago\AppData\Local\Temp\ipykernel_2872\2954685278.py", line 30, in pwm
    pwm_matrix = [{b: (sum(1 for seq in seqs if seq[i] == b) + pseudo) / (len(seqs) + pseudo * len(set(''.join(seqs))))
                 ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\tiago\AppData\Local\Temp\ipykernel_2872\2954685278.py", line 30, in <listcomp>
    pwm_matrix = [{b: (sum(1 for seq in seqs if seq[i] == b) + pseudo) / (len(seqs) + pseudo * len(set(''.join(seqs))))
                  ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\tiago\AppData\Lo

### PSSM (Position-Specific Scoring Matrix)

In [5]:
def pssm(seqs, pseudo=1):
    """
    Calcula a Matriz de Pontuação de Posição Específica (PSSM) para um conjunto de sequências

    Parâmetros:
    seqs : list[str]
        Lista de sequências

    pseudo : float
        Valor de pseudocontagens a ser adicionado para evitar problemas com probabilidades zero.

    Retorna:
    lista : list[dict[str, float]]
        Uma lista de dicionários que representa a PSSM
    """
    import math
    # Lista para armazenar os dicionários representando a PSSM
    lista = []
    bases = set(''.join(seqs))

    # Itera sobre as posições correspondentes em todas as sequências
    for pos in list(zip(*seqs)):
        # Dicionário para armazenar as pontuações para cada base na posição atual
        dicionario = {}
        
        # Calcula a pontuação para cada base usando a fórmula da PSSM
        for b in bases:
            # Fórmula da PSSM: log2((contagem da base + pseudocontagens) / (total de sequências + total de bases * pseudocontagens)) / 0.25
            dicionario[b] = round(math.log2((pos.count(b) + pseudo) / (len(seqs) + len(bases) * pseudo)) / 0.25, 2)

        # Adiciona o dicionário à lista
        lista.append(dicionario)

    # Retorna a lista de dicionários representando a PSSM
    return lista

Exemplo:

In [6]:
# Sequências de DNA fornecidas para a função pssm
seqs = ['ATTG', 'ATCG', 'ATTC', 'ACTC']

# Calcula a matriz PSSM com pseudocontagem igual a 1
matriz_pssm = pssm(seqs, pseudo=1)

# Cabeçalhos para a tabela, incluindo "Base" e as posições numeradas
headers = ["Base"] + [f"Posição {i+1}" for i in range(len(seqs[0]))]

# Construção da tabela usando uma compreensão de lista 
table = [[base] + [round(matriz_pssm[i][base] + 1, 3) for i in range(len(matriz_pssm))] for base in set(''.join(seqs))]

# Impressão da tabela usando a função tabulate
print(tabulate(table, headers))


Base      Posição 1    Posição 2    Posição 3    Posição 4
------  -----------  -----------  -----------  -----------
C            -11              -7           -7        -4.66
A             -1.71          -11          -11       -11
T            -11              -3           -3       -11
G            -11             -11          -11        -4.66


Testes:

In [7]:
import unittest

class TestPSSM(unittest.TestCase):

    def test_pssm(self):
        # Teste para sequências vazias
        self.assertEqual(pssm([]), [])

    def test_pssm_pseud_zero(self):
        # Teste para pseudocontagem de zero
        with self.assertRaises(ValueError):
            pssm(['CTA'], pseudo=0)

    def test_pssm_dif_zero(self):
        # Teste para pseudocontagens diferentes de zero
        self.assertEqual(pssm(['AT','AG'], pseudo=1), [{'A': -4.0, 'T': -10.34, 'C': -10.34, 'G': -10.34}, {'A': -10.34, 'T': -6.34, 'C': -10.34, 'G': -6.34}])
                         
# Executar os testes
result = unittest.TextTestRunner().run(unittest.TestLoader().loadTestsFromTestCase(TestPSSM))

# Verificar que falhas houveram nos testes
if not result.wasSuccessful():
    for failure in result.failures:
        print(failure)

.F.
FAIL: test_pssm_dif_zero (__main__.TestPSSM.test_pssm_dif_zero)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "C:\Users\tiago\AppData\Local\Temp\ipykernel_2872\3182518252.py", line 16, in test_pssm_dif_zero
    self.assertEqual(pssm(['AT','AG'], pseudo=1), [{'A': -4.0, 'T': -10.34, 'C': -10.34, 'G': -10.34}, {'A': -10.34, 'T': -6.34, 'C': -10.34, 'G': -6.34}])
AssertionError: Lists differ: [{'A': -2.95, 'T': -9.29, 'G': -9.29}, {'A': -9.2[22 chars].29}] != [{'A': -4.0, 'T': -10.34, 'C': -10.34, 'G': -10.3[50 chars].34}]

First differing element 0:
{'A': -2.95, 'T': -9.29, 'G': -9.29}
{'A': -4.0, 'T': -10.34, 'C': -10.34, 'G': -10.34}

- [{'A': -2.95, 'G': -9.29, 'T': -9.29}, {'A': -9.29, 'G': -5.29, 'T': -5.29}]
+ [{'A': -4.0, 'C': -10.34, 'G': -10.34, 'T': -10.34},
+  {'A': -10.34, 'C': -10.34, 'G': -6.34, 'T': -6.34}]

----------------------------------------------------------------------
Ran 3 tests in 0.005s

FA

(<__main__.TestPSSM testMethod=test_pssm_dif_zero>, 'Traceback (most recent call last):\n  File "C:\\Users\\tiago\\AppData\\Local\\Temp\\ipykernel_2872\\3182518252.py", line 16, in test_pssm_dif_zero\n    self.assertEqual(pssm([\'AT\',\'AG\'], pseudo=1), [{\'A\': -4.0, \'T\': -10.34, \'C\': -10.34, \'G\': -10.34}, {\'A\': -10.34, \'T\': -6.34, \'C\': -10.34, \'G\': -6.34}])\nAssertionError: Lists differ: [{\'A\': -2.95, \'T\': -9.29, \'G\': -9.29}, {\'A\': -9.2[22 chars].29}] != [{\'A\': -4.0, \'T\': -10.34, \'C\': -10.34, \'G\': -10.3[50 chars].34}]\n\nFirst differing element 0:\n{\'A\': -2.95, \'T\': -9.29, \'G\': -9.29}\n{\'A\': -4.0, \'T\': -10.34, \'C\': -10.34, \'G\': -10.34}\n\n- [{\'A\': -2.95, \'G\': -9.29, \'T\': -9.29}, {\'A\': -9.29, \'G\': -5.29, \'T\': -5.29}]\n+ [{\'A\': -4.0, \'C\': -10.34, \'G\': -10.34, \'T\': -10.34},\n+  {\'A\': -10.34, \'C\': -10.34, \'G\': -6.34, \'T\': -6.34}]\n')


### Probabilidade de uma sequência

In [8]:
def prob_seq(sequence, resultado):
    """
    Calcula a probabilidade de uma sequência utilizando como base na função de PWM

    Parâmetros:
    sequence : str 
        Sequência de DNA 

    resultado : list[dict[str, float]] 
        Variável que contém o resultado da função PWM, onde cada dicionário representa as probabilidades para cada 
        base numa posição

    Retorna:
    probabilidade : float 
        A probabilidade da sequência com base na PWM

    """    
    probabilidade = 1.0  # Inicializa a probabilidade como 1.0

    # Itera sobre cada posição e base na sequência
    for position, base in enumerate(sequence):
        # Verifica se a base está presente no resultado da PWM para essa posição
        if base in resultado[position]:
            # Multiplica a probabilidade acumulada pelo valor da PWM para a base nessa posição
            probabilidade *= resultado[position][base]
        else:
            # Se a base não estiver presente, atribui uma probabilidade mínima de 0.01
            probabilidade *= 0.01

    return probabilidade

Exemplo:

In [15]:
seq = 'ATAA'
probabilidade = prob_seq(seq, resultado)

print(f"A probabilidade da sequência '{seq}' é de: \n{probabilidade}")

A probabilidade da sequência 'ATAA' é de: 
0.0048828125


Testes:

In [10]:
import unittest  

class Test_prob_seq (unittest.TestCase):

    seqs = ['ATTG','ATCG','ATTC','ACTC']


    def test_prob_seq (self):
        self.assertEqual(prob_seq("ATTG",pwm(seqs, pseudo= 0)), 0.2845)

    def test_prob_seq (self):
        self.assertEqual(prob_seq("ATAA",pwm(seqs, pseudo= 0)), 0.0048828125)


# Executar os testes
result = unittest.TextTestRunner().run(unittest.TestLoader().loadTestsFromTestCase(Test_prob_seq))

# Verificar que falhas houveram nos testes
if not result.wasSuccessful():
    for failure in result.failures:
        print(failure)

F
FAIL: test_prob_seq (__main__.Test_prob_seq.test_prob_seq)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "C:\Users\tiago\AppData\Local\Temp\ipykernel_2872\930531534.py", line 12, in test_prob_seq
    self.assertEqual(prob_seq("ATAA",pwm(seqs, pseudo= 0)), 0.0048828125)
AssertionError: 0.0 != 0.0048828125

----------------------------------------------------------------------
Ran 1 test in 0.001s

FAILED (failures=1)


(<__main__.Test_prob_seq testMethod=test_prob_seq>, 'Traceback (most recent call last):\n  File "C:\\Users\\tiago\\AppData\\Local\\Temp\\ipykernel_2872\\930531534.py", line 12, in test_prob_seq\n    self.assertEqual(prob_seq("ATAA",pwm(seqs, pseudo= 0)), 0.0048828125)\nAssertionError: 0.0 != 0.0048828125\n')


### Sequencia mais provável

In [11]:
def seq_mais_prob(seqs, pseudo=0):
    """
    Função que permite devolver a sequência mais provável de ocorrer

    Parâmetros:
        seqs (list): Lista de sequências 
        pseudo_count (int, optional): Pseudocontagem. Defaults to 0.

    Retorna:
        str: Sequência mais provável de ocorrer
    """
    
    # Número total de letras (bases) no DNA
    num_letters = 4  # Assumindo que estamos a trabalhar com DNA (A, C, G, T)
    
    # Número de sequências fornecidas
    num_seqs = len(seqs)
    
    # Comprimento das sequências (assumindo que todas têm o mesmo comprimento)
    len_seqs = len(seqs[0])

    # Inicializa a matriz para armazenar contagens de frequência
    mat = [[0] * (len_seqs + 1) for _ in range(num_letters + 1)]
    resul = ""

    # Atualiza a matriz com contagens de frequência e aplica pseudocontagens
    for seq in seqs:
        for coluna in range(len_seqs):
            # Obtém a linha correspondente à base na sequência
            linha = "ACGT".index(seq[coluna])
            # Incrementa a contagem na célula correspondente
            mat[linha + 1][coluna + 1] += 1
            # Atualiza os rótulos nas linhas e colunas
            mat[0][coluna + 1] = f"  {coluna + 1} "
            mat[0][0] = "."
            mat[linha + 1][0] = "ACGT"[linha]

    # Normaliza as frequências para probabilidades
    for linha in range(1, num_letters + 1):
        for coluna in range(1, num_seqs + 1):
            # Aplica pseudocontagens e normaliza as frequências
            mat[linha][coluna] = round(float(mat[linha][coluna] + pseudo + 0.01) / float(num_seqs), 3)

    list_temp = []
    list_final = []

    # Encontra a posição mais provável para cada coluna
    for coluna in range(1, num_seqs + 1):
        for linha in range(1, num_letters + 1):
            # Adiciona as probabilidades e as linhas correspondentes numa lista temporária
            list_temp.append((mat[linha][coluna], linha))
        # Encontra a linha com a maior probabilidade e adiciona à lista final
        list_final.append(max(list_temp))
        list_temp = []

    # Constrói a sequência mais provável
    for c in list_final:
        for linha in range(1, len_seqs + 1):
            # Adiciona a base correspondente à linha mais provável à sequência resultante
            if linha == c[1]:
                resul += mat[linha][0]

    return resul

In [12]:
seq_mais_prob(seqs,pseudo=0)

'ATTG'

In [13]:
import unittest

class Test_seq_provavel(unittest.TestCase):

    def teste_1(self):
        self.assertEqual(seq_provavel('TAAA',pwm(['AAAA','AATA'])), 'TAAA')

    def teste_seq_invalid(self):
        with self.assertRaises(AssertionError):
            seq_provavel(["AAT", ],pwm(['AAA','TTT']))
    
    def test_seq_vazio(self):
        with self.assertRaises(AssertionError):
            seq_provavel(["", "ATC"],pwm(['AAA','TTT']))
                

# Executar os testes
result = unittest.TextTestRunner().run(unittest.TestLoader().loadTestsFromTestCase(Test_seq_provavel))

# Verificar que falhas houveram nos testes
if not result.wasSuccessful():
    for failure in result.failures:
        print(failure)

EEE
ERROR: test_seq_vazio (__main__.Test_seq_provavel.test_seq_vazio)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "C:\Users\tiago\AppData\Local\Temp\ipykernel_2872\316962021.py", line 14, in test_seq_vazio
    seq_provavel(["", "ATC"],pwm(['AAA','TTT']))
    ^^^^^^^^^^^^
NameError: name 'seq_provavel' is not defined

ERROR: teste_1 (__main__.Test_seq_provavel.teste_1)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "C:\Users\tiago\AppData\Local\Temp\ipykernel_2872\316962021.py", line 6, in teste_1
    self.assertEqual(seq_provavel('TAAA',pwm(['AAAA','AATA'])), 'TAAA')
                     ^^^^^^^^^^^^
NameError: name 'seq_provavel' is not defined

ERROR: teste_seq_invalid (__main__.Test_seq_provavel.teste_seq_invalid)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "C:\Users\tiago\AppData\L